In [1]:
import harris
import cv2
import numpy as np
from scipy.linalg import svd,lstsq

from matplotlib.pyplot import imshow

In [2]:
img1_orig = cv2.imread('./images/Limage.jpeg')
img2_orig = cv2.imread('./images/Rimage.jpeg')

In [3]:
img1 = cv2.cvtColor(img1_orig,cv2.COLOR_RGB2GRAY)
img2 = cv2.cvtColor(img2_orig,cv2.COLOR_RGB2GRAY)

In [4]:
#Blurring to reduce noise
img1 = cv2.GaussianBlur(img1,(3,3),0)
img2 = cv2.GaussianBlur(img2,(3,3),0)

#Increasing contrast by Histogram Equalisation
img1 = cv2.equalizeHist(img1)
img2 = cv2.equalizeHist(img2)

In [5]:
img1_corners,corners1 = harris.mark_corner_points(img1,5,5,0.04,True)
img2_corners,corners2 = harris.mark_corner_points(img2,5,5,0.04,True)

476
610


In [6]:
cv2.imwrite('image1.png',img1_corners)
cv2.imwrite('image2.png',img2_corners)

True

In [9]:
#Reading Common corner points
common_corner_list = []
with open('cornerlist.txt','r') as f:
    common_corner_list = f.readlines()
l_common_corner_list = [corner[:-1].split(' ')[:2] for corner in common_corner_list]
r_common_corner_list = [corner[:-1].split(' ')[2:] for corner in common_corner_list]
l_common_corner_list = np.array(l_common_corner_list,dtype=np.int32)
r_common_corner_list = np.array(r_common_corner_list,dtype=np.int32)

In [24]:
l_new_image = img1.copy()
l_new_image = cv2.cvtColor(l_new_image,cv2.COLOR_GRAY2RGB)

r_new_image = img2.copy()
r_new_image = cv2.cvtColor(r_new_image,cv2.COLOR_GRAY2RGB)

for corner in l_common_corner_list:
    cv2.circle(l_new_image,(int(corner[0]),int(corner[1])),4,(0,0,255),-1)
    
for corner in r_common_corner_list:
    cv2.circle(r_new_image,(int(corner[0]),int(corner[1])),4,(0,0,255),-1)

cv2.imwrite('Limage_commoncorners.png',l_new_image)
cv2.imwrite('Rimage_commoncorners.png',r_new_image)

True

In [28]:
#Finding homography matrix
homography_matrix , status = cv2.findHomography(l_common_corner_list,r_common_corner_list)

In [29]:
homography_matrix

array([[  1.24961539e+00,   2.01578464e-02,  -5.74777977e+02],
       [  2.38311603e-01,   1.19052608e+00,  -1.37854402e+02],
       [  3.14141502e-04,   1.23588045e-05,   1.00000000e+00]])

In [46]:
def stitch_image(img1,img2, homography_matrix):
    
    #Get dimensions of the input images
    r1,c1 = img1.shape[:2]
    r2,c2 = img2.shape[:2]
    
    #Canvas dimensions
    img1_canvas_dims = np.float32([ [0,0], [0,r1], [c1, r1], [c1,0] ]).reshape(-1,1,2)
    img2_canvas_dims = np.float32([ [0,0], [0,r2], [c2, r2], [c2,0] ]).reshape(-1,1,2)
    

    img1_canvas_dims = cv2.perspectiveTransform(img1_canvas_dims, homography_matrix)

    stitched_img_dims = np.concatenate( (img2_canvas_dims, img1_canvas_dims), axis = 0)
    
    
    [x_min, y_min] = np.int32(stitched_img_dims.min(axis=0).ravel())
    [x_max, y_max] = np.int32(stitched_img_dims.max(axis=0).ravel())
    
    
    transform_dist = [-x_min,-y_min]
    transform_array = np.array([[1, 0, transform_dist[0]], [0, 1, transform_dist[1]], [0,0,1]])
    
    stitched_img = cv2.warpPerspective(img1, transform_array.dot(homography_matrix), (x_max-x_min, y_max-y_min))
    stitched_img[transform_dist[1]:r1+transform_dist[1], transform_dist[0]:c1+transform_dist[0]] = img2
    
    #Cropping dimensions
    [x_min1, y_min1] = np.int32(img2_canvas_dims.min(axis=0).ravel())
    [x_max1, y_max1] = np.int32(img1_canvas_dims.max(axis=0).ravel())
    
    cropped = stitched_img[x_min:,y_min1 - y_min:y_max1-y_min]
    
    return stitched_img,cropped

In [47]:
stitched_img,cropped = stitch_image(img1_orig,img2_orig,homography_matrix)

In [48]:
cv2.imwrite('result.png',stitched_img)

True

In [61]:
mat = find_homography(l_common_corner_list,r_common_corner_list)

In [7]:
def find_affine_homography(points1,points2):
	if points1.shape[0] != points2.shape[0] : raise ValueError("The number of input and output points mismatches")
	if points1.shape[1] == 2 :
		p1 = np.ones((len(points1),3),'float64')
		p1[:,:2] = points1
	elif points1.shape[1] == 3 : p1 = points1
	else : raise ValueError("Bad shape for input points")
	
	if points2.shape[1] == 2 :
		p2 = np.ones((len(points2),3),'float64')
		p2[:,:2] = points2
	elif points2.shape[1] == 3 : p2 = points2
	else : raise ValueError("Bad shape for output points")
	
	npoints = len(points1)
	
	A = np.zeros((3*npoints,6),'float64')
	b = np.zeros((3*npoints,1),'float64')
	for i in xrange(npoints):
		p1i = p1[i]
		x2i,y2i,w2i = p2[i]
		xpi = x2i*p1i
		ypi = y2i*p1i
		wpi = w2i*p1i
		
		A[i*3  ,3:6] = -wpi
		A[i*3+1,0:3] =  wpi
		A[i*3+2,0:3] = -ypi
		A[i*3+2,3:6] =  xpi
		
		b[i*3  ] = -y2i*p1i[2]
		b[i*3+1] =  x2i*p1i[2]

	h = lstsq(A,b,overwrite_a = True, overwrite_b = True)[0]
	H = np.zeros( (3,3) , 'float64' )
	H[:2,:] = h.reshape(2,3)
	H[2,2] = 1
	return H

In [10]:
mat = find_affine_homography(l_common_corner_list,r_common_corner_list)

In [28]:
homography_matrix

array([[  1.24961539e+00,   2.01578464e-02,  -5.74777977e+02],
       [  2.38311603e-01,   1.19052608e+00,  -1.37854402e+02],
       [  3.14141502e-04,   1.23588045e-05,   1.00000000e+00]])

In [11]:
mat

array([[  3.51039784e-01,   1.51613773e-03,  -1.57947582e+02],
       [ -4.29115160e-02,   3.31689786e-01,   3.19106479e+01],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]])

In [69]:
rbhavan_corners,corners_rbhavan = harris.mark_corner_points(rbhavan,5,5,0.04,True)


235


In [70]:
cv2.imwrite('rbhavan_newcorners.png',rbhavan_corners)

True